In [57]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as F


spark = SparkSession \
            .builder \
            .config("spark.master", "local") \
            .config('spark.executor.cores', '8')\
            .config('spark.executor.memory', '2g')\
            .config('spark.driver.memory', '2g')\
            .config('spark.dynamicAllocation.minExecutors', '4')\
            .config("spark.jars", "/opt/conda/lib/python3.11/site-packages/pyspark/jars/clickhouse-jdbc-0.6.3-all.jar") \
            .appName("spark_halltape") \
    .getOrCreate()


print("Активные Spark сессии:", 'http://localhost:4040/jobs')

Активные Spark сессии: http://localhost:4040/jobs


In [75]:
spark.stop()

In [77]:
pwd

'/home/jovyan'

In [76]:
from pyspark.sql.functions import col

jdbc_driver = "com.clickhouse.jdbc.ClickHouseDriver"
host = "localhost"
port = "8123"
user = "admin"
password = "admin"
database = "raw"
options = "ssl=false"

jdbc_url = f"jdbc:clickhouse://{host}:{port}/{database}?{options}&user={user}&password={password}"

spark = (
    SparkSession
    .builder
    .appName("PoC-ClickHouse")
    .config("spark.jars", "/opt/conda/lib/python3.11/site-packages/pyspark/jars/clickhouse-spark-runtime-3.4_2.13-0.7.3.jar, /opt/conda/lib/python3.11/site-packages/pyspark/jars/clickhouse-jdbc-0.6.3-all.jar")
    .config("spark.driver.extraClassPath", "/opt/conda/lib/python3.11/site-packages/pyspark/jars/clickhouse-jdbc-0.6.3-all.jar")
    .config("spark.executor.extraClassPath", "/opt/conda/lib/python3.11/site-packages/pyspark/jars/clickhouse-jdbc-0.6.3-all.jar")
    .config("spark.sql.catalog.clickhouse.host", host)
    .config("spark.sql.catalog.clickhouse.protocol", "http")
    .config("spark.sql.catalog.clickhouse.http_port", port)  
    .config("spark.sql.catalog.clickhouse.user", user)
    .config("spark.sql.catalog.clickhouse.password", password)
    .config("spark.sql.catalog.clickhouse.database", database)
    .getOrCreate()
)

spark

df = spark.read.format("jdbc")\
        .option("url", jdbc_url)\
        .option("driver", jdbc_driver)\
        .option("dbtable", "table")\
        .load()

Py4JJavaError: An error occurred while calling o838.load.
: java.lang.ClassNotFoundException: com.clickhouse.jdbc.ClickHouseDriver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:41)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apa

method

In [59]:
spark.stop()

In [58]:

df = spark.read \
        .format("jdbc") \
        .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
        .option("url", "jdbc:clickhouse://localhost:9000") \
        .option("user", "admin") \
        .option("password", "admin") \
        .option("dbtable", "raw.my_table") \
        .load()

df.show()

Py4JJavaError: An error occurred while calling o583.load.
: java.lang.ClassNotFoundException: com.clickhouse.jdbc.ClickHouseDriver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:41)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apa

In [16]:
spark.stop()

In [14]:
path = '/home/jovyan'
date = '2024-07-01'


card = spark.read.csv(f"{path}/card.csv", header=True, sep=";")\
            .where(f''' load_date = "{date}" ''')

status_card = spark.read.csv(f"{path}/Status_card.csv", header=True, sep=";")\
                    .where(f''' load_date = "{date}"  ''')

transactions = spark.read.csv(f"{path}/transactions.csv", header=True, sep=";")\
                    .where(f''' load_date = "{date}"  ''')


In [3]:
first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx.show()

first_trx.count()

+----------------+--------------------+
|        card_num|transaction_datetime|
+----------------+--------------------+
|0005538167587579| 2024-07-01 02:25:10|
|0007178749480464| 2024-07-01 22:23:10|
|0071204528664757| 2024-07-02 00:01:10|
|0089135673710847| 2024-07-01 23:26:10|
|0092333514392220| 2024-07-01 14:55:10|
|0117477741827069| 2024-07-01 04:47:10|
|0123869799811751| 2024-07-01 08:09:10|
|0131542214103394| 2024-07-01 19:07:10|
|0140866230289371| 2024-07-01 20:22:10|
|0153027993228699| 2024-07-01 11:08:10|
|0162240485103303| 2024-07-01 07:36:10|
|0162466916966431| 2024-07-01 11:02:10|
|0192260687950694| 2024-07-01 23:43:10|
|0251171899513329| 2024-07-01 12:07:10|
|0255081869910908| 2024-07-01 08:02:10|
|0281764067659511| 2024-07-01 12:51:10|
|0299666908406242| 2024-07-01 21:30:10|
|0314501698532036| 2024-07-01 03:08:10|
|0317705894931047| 2024-07-01 10:22:10|
|0318703582249791| 2024-07-01 18:58:10|
+----------------+--------------------+
only showing top 20 rows



494

In [4]:
first_trx_info = first_trx.join(transactions, 'card_num', 'inner')

first_trx_info.where(''' card_num = "0005538167587579" ''').show()

+----------------+--------------------+------+--------------------+----------+
|        card_num|transaction_datetime|amount|transaction_datetime| load_date|
+----------------+--------------------+------+--------------------+----------+
|0005538167587579| 2024-07-01 02:25:10|498.32| 2024-07-01 02:37:10|2024-07-01|
|0005538167587579| 2024-07-01 02:25:10|  0.89| 2024-07-01 02:40:10|2024-07-01|
|0005538167587579| 2024-07-01 02:25:10|261.05| 2024-07-01 02:25:10|2024-07-01|
+----------------+--------------------+------+--------------------+----------+



In [5]:
first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
          
first_trx_info.where(''' card_num = "0005538167587579" ''').show()

+----------------+--------------------+------+----------+
|        card_num|transaction_datetime|amount| load_date|
+----------------+--------------------+------+----------+
|0005538167587579| 2024-07-01 02:25:10|261.05|2024-07-01|
+----------------+--------------------+------+----------+



In [6]:
dt_trx = first_trx_info.select('card_num', 'amount', '*')
df_st = status_card.select('card_num', 'card_num_md5', 'status')


result_df = dt_trx\
                .join(df_st, "card_num", "inner")\
                .join(card, "card_num_md5", "right")\
                .drop('card_num_md5')


total_rows = result_df.select('card_num').count()
total_unique_rows = result_df.select('card_num').distinct().count()

print(total_rows, total_unique_rows)

1007 495


In [7]:
result_df.where(''' amount IS NULL ''').show()

+--------+------+--------------------+------+---------+------+-------------------+--------------------+----------+----------+
|card_num|amount|transaction_datetime|amount|load_date|status|      card_order_dt|                 url|    cookie| load_date|
+--------+------+--------------------+------+---------+------+-------------------+--------------------+----------+----------+
|    null|  null|                null|  null|     null|  null|2024-07-01 00:34:00|http://example.co...|zj9q0BFbH0|2024-07-01|
|    null|  null|                null|  null|     null|  null|2024-07-01 09:20:00|http://example.co...|qxDEgcfeYL|2024-07-01|
|    null|  null|                null|  null|     null|  null|2024-07-01 17:55:00|http://example.co...|6N54ZrsTmQ|2024-07-01|
|    null|  null|                null|  null|     null|  null|2024-07-01 01:00:00|http://example.co...|FtQyCZ1hIn|2024-07-01|
|    null|  null|                null|  null|     null|  null|2024-07-01 16:58:00|http://example.co...|HH65JSJIFz|2024

In [8]:
result_df.where(''' card_num IS NOT NULL  ''').count()

501

In [9]:
result_df.where(''' card_num = "0005538167587579"  ''').show(truncate=False)

+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+
|card_num        |amount|transaction_datetime|amount|load_date |status|card_order_dt      |url                     |cookie    |load_date |
+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+
|0005538167587579|261.05|2024-07-01 02:25:10 |261.05|2024-07-01|выдана|2024-07-01 01:01:00|http://example.com/page1|0CCrvgl61G|2024-07-01|
+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+



In [10]:
result_df.where(''' card_num = "2070700245113982"  ''').show(truncate=False)

+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+
|card_num        |amount|transaction_datetime|amount|load_date |status|card_order_dt      |url                     |cookie    |load_date |
+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+
|2070700245113982|31.11 |2024-07-01 09:07:10 |31.11 |2024-07-01|выдана|2024-07-01 08:21:00|http://example.com/page4|NY45MJBknH|2024-07-01|
|2070700245113982|289.34|2024-07-01 09:07:10 |289.34|2024-07-01|выдана|2024-07-01 08:21:00|http://example.com/page4|NY45MJBknH|2024-07-01|
+----------------+------+--------------------+------+----------+------+-------------------+------------------------+----------+----------+



In [11]:
final_df = result_df\
                .where(''' card_num IS NOT NULL ''')\
                .groupBy('card_num',
                         'transaction_datetime',
                         'status',
                         'card_order_dt',
                         'url',
                         'cookie')\
                .agg(F.max('amount').alias('amt'))

final_df.show(truncate=False)

+----------------+--------------------+------+-------------------+-------------------------+----------+------+
|card_num        |transaction_datetime|status|card_order_dt      |url                      |cookie    |amt   |
+----------------+--------------------+------+-------------------+-------------------------+----------+------+
|0005538167587579|2024-07-01 02:25:10 |выдана|2024-07-01 01:01:00|http://example.com/page1 |0CCrvgl61G|261.05|
|0007178749480464|2024-07-01 22:23:10 |выдана|2024-07-01 22:05:00|http://example.com/page1 |zY81jXGIHg|241.46|
|0071204528664757|2024-07-02 00:01:10 |выдана|2024-07-01 23:39:00|http://example.com/page8 |WItT5ew8g7|379.81|
|0089135673710847|2024-07-01 23:26:10 |выдана|2024-07-01 22:08:00|http://example.com/page1 |qVxlIpM1Ow|86.43 |
|0092333514392220|2024-07-01 14:55:10 |выдана|2024-07-01 14:28:00|http://example.com/page6 |TEjKGfsZ9l|138.34|
|0117477741827069|2024-07-01 04:47:10 |выдана|2024-07-01 04:25:00|http://example.com/page2 |pTgqdowNRK|179.33|
|

In [12]:
datamart = final_df\
                .select('card_order_dt',
                        'card_num',
                        'cookie',
                        'url',
                        'amt',
                        'status')\
                .withColumn('transaction_level',
                                F.when(F.col('amt') > 300, True).otherwise(False))\
                .withColumn('status_flag',
                                F.when(F.col('status') == "выдана", True).otherwise(False))\
                .drop('amt', 'status')

datamart.show(truncate=False)

+-------------------+----------------+----------+-------------------------+-----------------+-----------+
|card_order_dt      |card_num        |cookie    |url                      |transaction_level|status_flag|
+-------------------+----------------+----------+-------------------------+-----------------+-----------+
|2024-07-01 01:01:00|0005538167587579|0CCrvgl61G|http://example.com/page1 |false            |true       |
|2024-07-01 22:05:00|0007178749480464|zY81jXGIHg|http://example.com/page1 |false            |true       |
|2024-07-01 23:39:00|0071204528664757|WItT5ew8g7|http://example.com/page8 |true             |true       |
|2024-07-01 22:08:00|0089135673710847|qVxlIpM1Ow|http://example.com/page1 |false            |true       |
|2024-07-01 14:28:00|0092333514392220|TEjKGfsZ9l|http://example.com/page6 |false            |true       |
|2024-07-01 04:25:00|0117477741827069|pTgqdowNRK|http://example.com/page2 |false            |true       |
|2024-07-01 08:06:00|0123869799811751|hJJgR36w

In [235]:
datamart.write.partitionBy('transaction_level').mode("overwrite").csv('datamart.csv')

In [40]:
spark.stop()

In [42]:
df = spark.read.csv('/Users/halltape/Desktop/CODE/test_code/spark_application/datasets/customs_data.csv')

df.rdd.getNumPartitions()

11

In [44]:
df.repartition(2).show()

24/07/22 18:17:08 WARN TaskSetManager: Lost task 3.0 in stage 2.0 (TID 5) (192.168.1.46 executor driver): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling o406.showString.
: org.apache.spark.SparkException: Job 2 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2785)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2660)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleStageCancellation$1(DAGScheduler.scala:2649)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:246)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:2642)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2937)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onRec

In [49]:
df.coalesce(2).rdd.glom().collect()

24/07/22 18:32:48 ERROR Executor: Exception in task 0.0 in stage 6.0 (TID 12) 2]
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$2740/486024616.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.$anonfun$writeBy

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 6 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1212)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1210)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1210)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3011)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2902)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1509)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2902)
	at org.apache.spark.SparkContext.$anonfun$stop$1

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58615)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/halltape/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/halltape/miniconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/halltape/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/Users/halltape/miniconda3/lib/python3.10/socketserve

### Check

In [120]:
first_trx_info.select('card_num').groupBy('card_num')\
            .agg(F.count('*').alias("total_rows"))\
            .orderBy(F.col('total_rows').desc()).show()




first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
          
first_trx_info.where(''' card_num = "2070700245113982" ''').show()



first_trx_info = first_trx.join(transactions, 'card_num', 'inner')
first_trx_info.where(''' card_num = "2070700245113982" ''').show()

+----------------+----------+
|        card_num|total_rows|
+----------------+----------+
|2070700245113982|         2|
|2262350645675539|         2|
|3103433078006244|         2|
|4348133075638360|         2|
|4544055876664150|         2|
|6242600491786411|         2|
|7787314985064344|         2|
|0005538167587579|         1|
|0007178749480464|         1|
|0071204528664757|         1|
|0089135673710847|         1|
|0092333514392220|         1|
|0117477741827069|         1|
|0123869799811751|         1|
|0131542214103394|         1|
|0140866230289371|         1|
|0153027993228699|         1|
|0162240485103303|         1|
|0162466916966431|         1|
|0192260687950694|         1|
+----------------+----------+
only showing top 20 rows

+----------------+--------------------+------+----------+
|        card_num|transaction_datetime|amount| load_date|
+----------------+--------------------+------+----------+
|2070700245113982| 2024-07-01 09:07:10| 31.11|2024-07-01|
|2070700245113982| 202